# Módulo 6 - Aprendizaje profundo
## Clase 3: RNN

### Parámetros y Hiperparámetros  modelo RNN

A continuación se describen los principales parámetros y los hiperparámetros que se pueden ajustar en el modelo LSTM (RNN):

#### 1. **vocab_size**
- **Descripción**: El tamaño del vocabulario utilizado por el modelo. Representa el número de caracteres únicos que el modelo puede reconocer y generar.
- **Importancia**: Cuanto más grande sea el vocabulario, más diverso puede ser el texto generado. Sin embargo, vocabularios más grandes pueden aumentar la complejidad del modelo.

#### 2. **embed_size**
- **Descripción**: Tamaño de la capa de embeddings. Los embeddings transforman los caracteres en vectores de características que capturan relaciones entre ellos.
- **Importancia**: Un tamaño mayor permite representar más información en cada vector de caracteres. Sin embargo, tamaños demasiado grandes pueden aumentar el tiempo de entrenamiento.

#### 3. **hidden_size**
- **Descripción**: El número de unidades en las capas ocultas (hidden layers) de la LSTM.
- **Importancia**: Determina la capacidad de la red para aprender patrones complejos. Un valor alto permite que el modelo capture más detalles, pero puede llevar a tiempos de entrenamiento más largos.

#### 4. **num_layers**
- **Descripción**: El número de capas LSTM apiladas en el modelo.
- **Importancia**: Aumentar el número de capas puede permitir que el modelo capture estructuras más complejas, pero también puede llevar a problemas como el desvanecimiento del gradiente (vanishing gradients) o sobreajuste (overfitting) si no se maneja adecuadamente.

#### 5. **dropout**
- **Descripción**: Tasa de desactivación de neuronas durante el entrenamiento para prevenir el sobreajuste.
- **Importancia**: Valores más altos de dropout (ej. 0.3 a 0.5) pueden ayudar a reducir el sobreajuste, pero tasas muy altas pueden afectar el rendimiento del modelo.

#### 6. **activation**
- **Descripción**: La función de activación que se aplica después de la LSTM. Puede ser `relu`, `tanh`, o ninguna (si se especifica `Identity`).
- **Importancia**: `relu` es una función común en redes neuronales debido a su eficiencia en resolver problemas de gradiente, mientras que `tanh` puede ser más adecuada para capturar relaciones no lineales en datos secuenciales como el texto.

#### 7. **batch_size**
- **Descripción**: Número de ejemplos que se pasan al modelo antes de actualizar los parámetros.
- **Importancia**: Batch sizes más grandes pueden acelerar el entrenamiento al aprovechar mejor la paralelización, mientras que batch sizes más pequeños pueden ofrecer mayor precisión en la actualización de los parámetros.

#### 8. **epochs**
- **Descripción**: Número de veces que el conjunto de datos completo pasa por la red durante el entrenamiento.
- **Importancia**: Más épocas permiten al modelo aprender mejor los patrones, pero demasiadas épocas pueden llevar a sobreajuste.

#### 9. **learning_rate**
- **Descripción**: Tasa de aprendizaje del modelo, que determina el tamaño de los pasos que el optimizador da durante el ajuste de los parámetros.
- **Importancia**: Un valor de tasa de aprendizaje bajo puede hacer que el entrenamiento sea muy lento, mientras que un valor demasiado alto puede hacer que el modelo no converja correctamente.


# 1) Ejemplo 1

En este ejemplo, se utiliza una Red Neuronal Recurrente (RNN) para aprender a generar texto basado en un conjunto de refranes en español.


### Paso 1: Crear el dataset de refranes
Inicia con una lista de refranes en español, que se prepara para que el modelo aprenda patrones de frases y palabras.

### Paso 2: Preprocesamiento del texto
Cada refrán se convierte en secuencias de palabras. Luego, se crean diccionarios que asignan a cada palabra un índice numérico, lo cual es esencial para el procesamiento en la RNN.

### Paso 3: Convertir el texto en secuencias de índices
El texto se transforma en secuencias numéricas, donde cada palabra se representa por un número. Esto ayuda al modelo a identificar y aprender patrones a nivel de palabras.

### Paso 4: Definir el modelo RNN
El modelo tiene tres componentes: una capa de embedding (convierte palabras en vectores), una capa LSTM (captura patrones en las secuencias) y una capa de salida (predice la siguiente palabra en la secuencia).

### Paso 5: Función de pérdida y optimizador
La función de pérdida mide el error del modelo, y el optimizador ajusta sus parámetros para mejorar su rendimiento en cada iteración.

### Paso 6: Configuración del dispositivo
Para mejorar la velocidad de entrenamiento, el modelo y los datos se transfieren a la GPU, si está disponible.

### Paso 7: Entrenamiento del modelo
El modelo se entrena en múltiples ciclos (épocas), donde aprende a mejorar sus predicciones en cada pasada de los datos. Se imprime la pérdida promedio para evaluar el aprendizaje del modelo.

### Paso 8: Generación de texto a partir de una semilla
Usando una frase inicial, el modelo genera una secuencia de palabras al predecir cada palabra siguiente en base a las anteriores. Así, se crean nuevas frases o refranes.





In [ ]:
# Ejemplo 1: Generación de texto con una RNN simple usando refranes en español a nivel de palabras

import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Paso 1: Crear un dataset ampliado de refranes en español
refranes = [
    "Al mal tiempo, buena cara.",
    "Más vale tarde que nunca.",
    "No por mucho madrugar amanece más temprano.",
    "A caballo regalado no se le mira el diente.",
    "El que guarda su código, siempre encuentra respaldo.",
    "El que mucho abarca poco aprieta.",
    "En casa de herrero, cuchillo de palo.",
    "A quien madruga, Dios le ayuda.",
    "No hay mal que por bien no venga.",
    "Ojos que no ven, corazón que no siente.",
    "Más vale pájaro en mano que cientos volando.",
    "No por mucho entrenar se logra generalizar.",
    "A palabras necias, oídos sordos.",
    "¿Tu GPU está al 100%? Mejor baja el batch antes de la combustión.",
    "Dime cómo entrenas y te diré cómo predices.",
    "El hábito no hace al monje.",
    "Dime con quién andas y te diré quién eres.",
    "Más sabe el diablo por viejo que por diablo.",
    "En boca cerrada no entran moscas.",
    "En las clases de ciencia de datos, el que lee a Bishop, hallara la respuesta.",
    "Cría cuervos y te sacarán los ojos.",
    "A buen entendedor, pocas palabras bastan.",
    "Perro que ladra no muerde.",
    "No hay peor ciego que el que no quiere ver.",
    "Al que madruga, Dios le ayuda.",
    "Al estudiante que duda, su código le ayuda.",
    "Más vale un buen 'debug' que diez errores ocultos.",
    "Quien mucho 'importa', al final poco encuentra.",
    "En las clases de ciencia de datos, el que pregunta mucho, aprende más.",
    "Cuando el código falla, todos dicen: ¡era el dataset!",
    "No hay peor frustración que ver el error en la última ejecución.",
    "A los datos duros, modelo con más parámetros.",
    "Si te quedas sin RAM,, mejor ponte a muestrear.",
    "Alumno que no entrena, no mejora su red.",
    "Divide y vencerás, dijo el que hacía cross-validation."
]

# Paso 2: Preprocesamiento del texto a nivel de palabras
texto = ' '.join(refranes).lower()
palabras = texto.split()

# Crear un conjunto de palabras únicas y definir el tamaño del vocabulario
palabras_unicas = sorted(list(set(palabras)))
vocab_size = len(palabras_unicas)

# Crear diccionarios para mapear palabras a índices y viceversa
word_to_idx = {word: i for i, word in enumerate(palabras_unicas)}
idx_to_word = {i: word for i, word in enumerate(palabras_unicas)}

print(f"Total de palabras únicas (vocab_size): {vocab_size}")

# RECORDAR VAMOS A PREDECIR LA SIGUIENTE PALABRA DEL REFRAN

# Paso 3: Convertir el texto a secuencias de índices
texto_idx = [word_to_idx[word] for word in palabras]
seq_length = 5
step = 1 #  serie continua y solapada

# Crear secuencias de entrada y sus palabras siguientes
sequences = []
next_words = []

for i in range(0, len(texto_idx) - seq_length, step):
    sequences.append(texto_idx[i: i + seq_length])
    next_words.append(texto_idx[i + seq_length])

print(f"Número de secuencias generadas: {len(sequences)}")

# Convertir secuencias a tensores de PyTorch int64
X = torch.tensor(sequences, dtype=torch.long)
y = torch.tensor(next_words, dtype=torch.long)

# Ejemplo: seq_length = 4
# X: ["Perro", "que", "ladra", "no"]   =   [12, 36, 38, 50]
# y: ["muerde"]                        =   [14]

# Dividir el dataset en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear DataLoader para entrenamiento y validación
batch_size = 16 # recuerden el seq_length entonces tendremos (16, 5)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Tamaño de X: {X.shape}")
print(f"Tamaño de y: {y.shape}")


# Parámetros del modelo
embed_size = 128  # dimensión del vector para cada palabra
# ejemplo embed_size = 3 , estos son los valores pesos que se ajustaran
#"perro"    = [0.8, 0.1, 0.5]
#"ladra"    = [0.7, 0.2, 0.5]
#"cuchillo" = [0.1, 0.8, 0.3]

hidden_size = 256  # neuronas en la capa interna de la LSTM, son la "memoria" de la red
# ejemplo hidden_size = 3 ,
#[0.6, 0.5, 0.4], "perro", "ladra", "cuchillo"(es el ultimo pesa menos)


# Paso 4: Definir la arquitectura del modelo RNN a nivel de palabras
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size # tamaño capa oculta
        self.embedding = nn.Embedding(vocab_size, embed_size) #(palabras unicas, dimensión del vector palabra)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True) # I/O (batch_size, seq_length, embed_size)
        self.fc = nn.Linear(hidden_size, vocab_size) # i: salida de la capa oculta (memoria), O: puntuaciones para cada palabra del vocabulario

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])  # bash, palabras(ultima palabra), embedding
        return out, hidden # devolvemos la predicion y la memoria

    def init_hidden(self, batch_size): # inicia estado oculto
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size)) # (h_0, c_0) corto y largo plazo

model = SimpleRNN(vocab_size, embed_size, hidden_size)

# Paso 5: Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Paso 6: Configuración del dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Entrenar el modelo usando DataLoader
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        hidden = model.init_hidden(X_batch.size(0)) # inicializo por bash
        hidden = tuple([h.to(device) for h in hidden]) # Mueve el estado oculto (h_0, c_0)

        optimizer.zero_grad()
        output, hidden = model(X_batch, hidden)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    if (epoch + 1) % 10 == 0:
        print(f"Época {epoch + 1}/{num_epochs}, Pérdida promedio de entrenamiento: {avg_train_loss:.4f}")

# Paso 7: Función para generar texto a partir de una semilla
def generar_texto(model, seed_text, length=20):
    model.eval()
    generated = seed_text
    words = seed_text.lower().split()
    hidden = model.init_hidden(1)
    hidden = tuple([h.to(device) for h in hidden])

    # Convertir las palabras de la semilla a índices
    input_seq = torch.tensor([[word_to_idx.get(word, 0) for word in words]], dtype=torch.long).to(device)

    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        prob = output.softmax(dim=1).data # recuerden recibimos las probabilidas asociadas a nuestro vocabulario
        word_idx = torch.multinomial(prob, 1).item() # seleciona 1 de las palabras más relevantes
        word = idx_to_word[word_idx]
        generated += ' ' + word

        # Actualizar la secuencia de entrada con la nueva palabra
        input_seq = torch.tensor([[word_idx]], dtype=torch.long).to(device)

    return generated

# Paso 8: Probar la generación de texto a partir de una semilla
seed_text = "Más vale pájaro en mano"
texto_generado = generar_texto(model, seed_text, length=5)

print("\nTexto generado:\n")
print(texto_generado)


Total de palabras únicas (vocab_size): 162
Número de secuencias generadas: 275
Tamaño de X: torch.Size([275, 5])
Tamaño de y: torch.Size([275])
Época 10/50, Pérdida promedio de entrenamiento: 0.2970
Época 20/50, Pérdida promedio de entrenamiento: 0.0513
Época 30/50, Pérdida promedio de entrenamiento: 0.0288
Época 40/50, Pérdida promedio de entrenamiento: 0.0221
Época 50/50, Pérdida promedio de entrenamiento: 0.0178

Texto generado:

Más vale pájaro en mano que cientos no entrena, que


In [ ]:
# Paso 10: Probar la generación de texto a partir de una semilla
seed_text = "En casa de herrero,"
texto_generado = generar_texto(model, seed_text, length=5)

print("\nTexto generado:\n")
print(texto_generado)


Texto generado:

En casa de herrero, cuchillo de palo. de palo.


In [ ]:
# TODO 1: Cambia el batch_size y num_epochs para optimizar el modelo

# TODO 2: # Cambia el `learning_rate` en el optimizador.
# ¿Qué observas cuando reduces o aumentas la tasa de aprendizaje? ¿Se estabiliza la pérdida más rápido?

# TODO 3: Cambia `embed_size` y `hidden_size` en la definición del modelo.
# ¿Cómo afecta el tamaño de embedding en el aprendizaje del modelo? ¿Y el tamaño de la capa oculta?

# TODO 4:  Cambia `seq_length` en la generación de secuencias.
# Pregunta: ¿La longitud de la secuencia afecta la coherencia del texto generado? ¿Por qué puede ser relevante?

# TODO 5: Cambia el valor de `seed_text` en la función `generar_texto`.
# ¿Cómo influye el texto inicial en las predicciones de la red? ¿La semilla afecta la coherencia del texto generado?

# TODO 6: Agrega más refranes o crea variaciones de los refranes existentes.
# ¿Crees que un conjunto de datos más grande ayudará al modelo a generar texto más coherente? ¿Por qué?


# 2) Ejemplo 2

Este ejemplo utiliza una Red Neuronal Recurrente (RNN) para generar texto basado en poemas en español, descargados de Project Gutenberg. El proceso sigue estos pasos:

1. **Carga y Preparación de los Poemas**:
   Se descarga el archivo de texto y se extraen los poemas de un rango de líneas específico. Cada poema se almacena en un diccionario para su fácil acceso y procesamiento.

2. **Preprocesamiento del Texto**:
   Todo el texto se convierte en una cadena única. Se crea un vocabulario de caracteres únicos, y se asigna un índice numérico a cada carácter. Esto permite que el modelo trabaje con representaciones numéricas.

3. **Conversión a Secuencias de Entrenamiento**:
   El texto se divide en secuencias de una longitud fija de caracteres, con el carácter siguiente como objetivo a predecir. Estas secuencias se convierten a tensores para PyTorch.

4. **Definición del Modelo RNN (Modelo 1)**:
   El modelo RNN incluye:
   - Una capa de embeddings para convertir caracteres en vectores.
   - Una capa LSTM para capturar patrones en secuencias.
   - Una capa final para predecir el siguiente carácter.

5. **Modelo Personalizable (Modelo 2)**:
   Una versión del modelo permite cambiar parámetros como el tamaño de la capa oculta y el número de capas LSTM. Esto ayuda a comparar el impacto de diferentes configuraciones.

6. **Entrenamiento del Modelo**:
   Se entrena el modelo usando lotes de datos y un optimizador que ajusta sus parámetros. Se muestra la pérdida promedio en cada época, indicando el progreso del aprendizaje.

7. **Generación de Texto con el Modelo**:
   Partiendo de una frase inicial, el modelo genera texto carácter por carácter. Cada nuevo carácter generado se usa como entrada en el siguiente paso, extendiendo el texto a partir de la frase inicial.

Poemas o cuentos en español:
dataset de letras de poemas disponibles en Project Gutenberg.

Ejemplo:
Machado, Antonio, 1875-1939
Páginas escogidas (Spanish) (as Author)
Poesías completas (Spanish) (as Author)

https://www.gutenberg.org/ebooks/68525


In [ ]:
import pandas as pd
import numpy as np
import string
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm  # Para mostrar bar
import re  # Para expresiones regulares
import matplotlib.pyplot as plt
import random
import os
import time

In [ ]:
#@title Esta celda se encarga de leer y procesa un archivo de texto para extraer poemas dentro de un rango de líneas dado.

!wget https://www.gutenberg.org/cache/epub/68525/pg68525.txt -O machado.txt

import re

def procesar_poemas(ruta_archivo, inicio, fin):
    # Paso 1: Leer todas las líneas del archivo
    print("Leyendo el archivo...")
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Verificar si el rango es válido
    if inicio < 0 or fin > len(lines):
        raise ValueError("El rango de líneas es inválido para el archivo dado.")

    # Paso 2: Extraer solo las líneas dentro del rango especificado
    print(f"Extrayendo líneas desde {inicio} hasta {fin}...")
    poemas_lineas = lines[inicio:fin]

    # Crear un diccionario para almacenar los títulos y contenido de los poemas
    poemas_dict = {}

    # Variables temporales para el título actual y el contenido del poema
    titulo_actual = None
    poema_actual = []

    # Paso 3: Procesar cada línea dentro del rango para extraer los títulos y contenido
    print("Procesando líneas para extraer títulos y poemas...")
    for i in range(len(poemas_lineas)):
        linea = poemas_lineas[i].strip()

        # Detectar si la línea contiene un número romano (presumiblemente indica un nuevo poema)
        if re.match(r'^[IVXLCDM]+$', linea):
            # Guardar el poema anterior si ya tenemos un título
            if titulo_actual and poema_actual:
                poemas_dict[titulo_actual] = '\n'.join(poema_actual).strip()
                poema_actual = []  # Reiniciar para el próximo poema

            # Paso 4: Identificar el nuevo título en la línea siguiente
            titulo_actual = poemas_lineas[i + 1].strip()
            i += 1  # Saltar la línea del título ya procesada

        # Paso 5: Agregar líneas al contenido del poema si estamos en un poema actual
        elif titulo_actual:
            poema_actual.append(linea)

    # Paso 6: Guardar el último poema en el diccionario si existe
    if titulo_actual and poema_actual:
        poemas_dict[titulo_actual] = '\n'.join(poema_actual).strip()

    # Imprimir el primer poema para verificar el resultado
    print("Procesamiento completo.")
    if poemas_dict:
        primer_titulo = list(poemas_dict.keys())[0]
        print(f"Título del primer poema: {primer_titulo}")
        print(f"Contenido del poema:\n{poemas_dict[primer_titulo][:500]}")
    else:
        print("No se encontraron poemas en el rango especificado.")

    # Listar los títulos de todos los poemas encontrados
    print("\nListado de títulos de los poemas encontrados:\n")
    for i, title in enumerate(poemas_dict.keys(), start=1):
        print(f"{i}. {title}")

    return poemas_dict

# Parámetros de entrada
ruta_archivo = 'machado.txt'  # Ruta del archivo a procesar
linea_inicio = 490  # Línea inicial
linea_fin = 7363    # Línea final

# Llamar a la función con los parámetros especificados
poems_dict = procesar_poemas(ruta_archivo, linea_inicio, linea_fin)


--2024-11-07 13:03:51--  https://www.gutenberg.org/cache/epub/68525/pg68525.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261733 (256K) [text/plain]
Saving to: ‘machado.txt’

machado.txt         100%[===================>] 255.60K  1.01MB/s    in 0.2s    

2024-11-07 13:03:52 (1.01 MB/s) - ‘machado.txt’ saved [261733/261733]

Leyendo el archivo...
Extrayendo líneas desde 490 hasta 7363...
Procesando líneas para extraer títulos y poemas...
Procesamiento completo.
Título del primer poema: EL VIAJERO
Contenido del poema:
EL VIAJERO

Está en la sala familiar, sombría,
y entre nosotros, el querido hermano
que en el sueño infantil de un claro día
vimos partir hacia un país lejano.

Hoy tiene ya las sienes plateadas,
un gris mechón sobre la angosta frente;
y la fría inquietud de sus miradas
revela un 

In [ ]:
#@title Esta celda convierte el texto en secuencias numéricas de longitud fija para que el modelo aprenda patrones en el texto.

import torch
import numpy as np

# Paso 1: Concatenar todos los poemas en un solo bloque de texto
# Aquí unimos todos los poemas en una cadena larga para analizar el texto completo.
all_poems_text = ' '.join(poems_dict.values())

# Paso 2: Crear un conjunto único de caracteres
# Esto obtiene todos los caracteres únicos presentes en el texto. El tamaño del conjunto será el vocabulario.
chars = sorted(list(set(all_poems_text)))
vocab_size = len(chars)

# Paso 3: Crear diccionarios de mapeo de caracteres a índices y viceversa
# Estos diccionarios permiten convertir caracteres en índices y viceversa, facilitando el procesamiento en el modelo.
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

# Paso 4: Convertir el texto a una secuencia de índices
# Transformamos cada carácter del texto en su índice correspondiente según el diccionario `char_to_idx`.
text_as_int = np.array([char_to_idx[char] for char in all_poems_text])

# Paso 5: Definir la longitud de las secuencias y el paso entre ellas
SEQ_LENGTH = 100  # Longitud de cada secuencia de entrada al modelo
step = 1          # Número de caracteres que avanzamos para generar la siguiente secuencia

# Paso 6: Crear listas para las secuencias y los caracteres siguientes
# En cada paso, generamos una secuencia de longitud `SEQ_LENGTH` y almacenamos el carácter que sigue a esa secuencia.
sequences = []
next_chars = []

# Bucle para generar secuencias y sus caracteres siguientes
for i in range(0, len(text_as_int) - SEQ_LENGTH, step):
    sequences.append(text_as_int[i:i + SEQ_LENGTH])       # Extraemos una secuencia de `SEQ_LENGTH` caracteres
    next_chars.append(text_as_int[i + SEQ_LENGTH])        # Guardamos el carácter que sigue a la secuencia

# Paso 7: Convertir las listas de secuencias y caracteres a tensores de PyTorch
# `X` contiene las secuencias de entrada y `y` contiene el carácter objetivo a predecir para cada secuencia.
X = torch.tensor(sequences, dtype=torch.long)
y = torch.tensor(next_chars, dtype=torch.long)

# Paso 8: Imprimir información sobre el procesamiento
print(f"Tamaño del vocabulario: {vocab_size}")  # Tamaño del conjunto de caracteres únicos
print(f"Número de secuencias generadas: {X.size(0)}")  # Número total de secuencias creadas para el entrenamiento


Tamaño del vocabulario: 84
Número de secuencias generadas: 80022


<ipython-input-6-b409516811ac>:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X = torch.tensor(sequences, dtype=torch.long)


In [ ]:
#@title Modelo 1
#Este código define una red neuronal recurrente (RNN). Este modelo usa embeddings y una capa LSTM

# Este código define una red neuronal recurrente (RNN) con embeddings y una capa LSTM, con opciones para añadir capas adicionales.

import torch.nn as nn

class CustomizableRNN(nn.Module):
    def __init__(self, vocab_size, embed_size=64, hidden_size=256, num_layers=1, dropout=0, activation='relu'):
        super(CustomizableRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Capa de embeddings
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # LSTM personalizada
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)

        # Capa fully connected (salida)
        self.fc = nn.Linear(hidden_size, vocab_size)

        # Función de activación
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:
            self.activation = nn.Identity()  # Sin activación si no se especifica

        # Opcional capas adicionales:
        # self.fc1_adicional = nn.Linear(hidden_size, hidden_size)   # Capa fully connected adicional
        # self.activation_fc1 = nn.ReLU()                            # Activación ReLU para capa adicional
        # self.fc2_adicional = nn.Linear(hidden_size, hidden_size)   # Segunda capa fully connected adicional
        # self.activation_fc2 = nn.ReLU()                            # Activación ReLU para segunda capa

    def forward(self, x, h):
        # Paso a través de la capa de embeddings
        x = self.embedding(x)

        # Paso por LSTM
        out, h = self.lstm(x, h)

        # Aplicar función de activación (si se especificó)
        out = self.activation(out[:, -1, :])

        # Paso por capas adicionales si se descomentan
        # out = self.fc1_adicional(out)             # Paso por primera capa fully connected adicional
        # out = self.activation_fc1(out)            # Aplicar activación ReLU
        # out = self.fc2_adicional(out)             # Paso por segunda capa fully connected adicional
        # out = self.activation_fc2(out)            # Aplicar activación ReLU

        # Paso final por la capa fully connected de salida
        out = self.fc(out)
        return out, h

    def init_hidden(self, batch_size):
        # Inicializar el estado oculto (hidden state y cell state) con ceros
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))

# Ejemplo de hiperparámetros
embed_size = 128       # Tamaño del embedding
hidden_size = 512      # Tamaño de la capa oculta
num_layers = 2         # Número de capas LSTM
dropout = 0.3          # Dropout para evitar sobreajuste
activation = 'tanh'    # Función de activación

# Crear el modelo con los hiperparámetros personalizables
model = CustomizableRNN(vocab_size, embed_size, hidden_size, num_layers, dropout, activation)

# Imprimir el modelo para verificar la estructura
print(model)



CustomizableRNN(
  (embedding): Embedding(84, 128)
  (lstm): LSTM(128, 512, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=512, out_features=84, bias=True)
  (activation): Tanh()
)


In [ ]:
# Función de entrenamiento con cálculo de precisión
def train(model, X, y, epochs=5, batch_size=64):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        # Bucle de entrenamiento
        for i in range(0, len(X) - batch_size, batch_size):
            inputs = X[i:i+batch_size]
            targets = y[i:i+batch_size]

            # Inicializar el estado oculto en el dispositivo correcto
            h = model.init_hidden(batch_size)
            h = tuple([each.to(device) for each in h])  # Asegurar que h esté en el mismo dispositivo que el modelo

            # Reseteo del optimizador y desconectar el gradiente del estado oculto
            optimizer.zero_grad()
            h = tuple([each.data for each in h])

            # Forward pass
            output, h = model(inputs, h)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            # Acumular pérdida total
            total_loss += loss.item()

            # Calcular precisión
            preds = output.argmax(dim=1)  # Predicción con mayor probabilidad
            correct_predictions += (preds == targets).sum().item()  # Comparar predicciones correctas
            total_samples += targets.size(0)

        avg_loss = total_loss / (len(X) // batch_size)
        accuracy = correct_predictions / total_samples
        print(f"Epoch [{epoch+1}/{epochs}], Pérdida promedio: {avg_loss:.4f}, Precisión: {accuracy:.4f}")

print("Función de entrenamiento creada ")


Función de entrenamiento creada 


In [ ]:
### modelo 1

# recordar si lo hacemos muy grande, se nos cuelga la maquina
batch_size = 32
epochs = 10
learning_rate = 0.001

# Inicialización de la función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
X, y = X.to(device), y.to(device)

# Entrenar el modelo
train(model, X, y, epochs=epochs, batch_size=batch_size)


Epoch [1/10], Pérdida promedio: 2.2370, Precisión: 0.3286
Epoch [2/10], Pérdida promedio: 1.9205, Precisión: 0.4068
Epoch [3/10], Pérdida promedio: 1.7772, Precisión: 0.4487
Epoch [4/10], Pérdida promedio: 1.6668, Precisión: 0.4809
Epoch [5/10], Pérdida promedio: 1.5765, Precisión: 0.5078
Epoch [6/10], Pérdida promedio: 1.4996, Precisión: 0.5295
Epoch [7/10], Pérdida promedio: 1.4348, Precisión: 0.5485
Epoch [8/10], Pérdida promedio: 1.3753, Precisión: 0.5648
Epoch [9/10], Pérdida promedio: 1.3260, Precisión: 0.5787
Epoch [10/10], Pérdida promedio: 1.2772, Precisión: 0.5923


In [ ]:
def generate_text(model, seed_text, length=300):
    model.eval()  # Cambiar a modo evaluación
    generated = seed_text
    h = model.init_hidden(1)  # Estado oculto inicial con batch_size = 1
    h = tuple([each.to(device) for each in h])  # Asegurarse de que el estado oculto esté en el dispositivo correcto

    # Convertir la semilla a índices
    input_seq = torch.tensor([char_to_idx[char] for char in seed_text], dtype=torch.long).unsqueeze(0).to(device)

    for _ in range(length):
        output, h = model(input_seq, h)
        prob = output.softmax(dim=1).data
        char_idx = torch.multinomial(prob, 1).item()
        char = idx_to_char[char_idx]
        generated += char

        # Usar el nuevo carácter generado como entrada para el próximo paso
        input_seq = torch.tensor([[char_idx]], dtype=torch.long).to(device)

    return generated
print("Funcion generar texto creada")

Funcion generar texto creada


In [ ]:
# Generar un nuevo poema a partir de una semilla con el modelo 1
seed_text = "El viento susurra "
generated_poem = generate_text(model, seed_text.lower(), length=300)
print("Poema generado modelo :\n")
print(generated_poem)


Poema generado modelo :

el viento susurra hacha
de España del que de un lameo.

Tiene el jardín al ángel viento
del agua de Ciruz, lleva la sombra paz sin púmo del ver manchito.

Huye del viejo ahuyente al fon y la guerra,
aldamas y aquí, hoy ayer, y Azorín,
paba era quiere Marís.

(Bonzario el águila a mismo.
La bunda quiere, parda el mar 


In [ ]:
# TODO 1: Modificar el número de épocas en ambos modelos.
# ¿Observas que aumentar las épocas siempre mejora el texto generado, o llega un punto en el que la calidad deja de mejorar?

# TODO 2: Aplcia "early stopping" .
# ¿Cuántas épocas sin mejoras necesitas para detener el entrenamiento? ¿Crees que esto mejora la eficiencia sin comprometer la calidad?

# TODO 3: Ajusta la longitud de las secuencias (SEQ_LENGTH).
# ¿Una longitud de secuencia más larga o más corta produce un texto más coherente?

# TODO 4: Cambia la función de activación en el CustomizableRNN (Modelo 2) entre relu, tanh, y Identity
# ¿Qué función de activación parece capturar mejor el estilo de los poemas? ¿Por qué crees que ocurre esto?

# TODO 5: Disminuye el tamaño de hidden_size en ambos modelos
# ¿El modelo sigue generando texto coherente con una capa oculta más pequeña? ¿En qué cambia la calidad del texto?

# TODO 6: Añade una capa de dropout en el SimpleRNN y observa el efecto en la generalización del modelo.
# ¿Cómo afecta el dropout al texto generado? ¿Notas menos repetición en el texto, o algún cambio en la coherencia?

# TODO 7: Cambia la frase inicial de generación de texto (seed_text y analiza cómo afecta el estilo y coherencia del texto generado.
# ¿La semilla inicial tiene un impacto importante en el estilo del texto? ¿Qué pasa si pruebas con una frase fuera del tema de los poemas?

# TODO 8: Cambia el batch_size para observar el impacto en la velocidad de entrenamiento y en la pérdida final.
# ¿Entrenar con un tamaño de batch más pequeño afecta la convergencia de la pérdida y la calidad del texto?

# TODO 9:  Ajusta la longitud de texto generado (length)
# ¿El modelo logra mantener coherencia en frases más largas? ¿Cómo cambia el estilo y la coherencia según la longitud?

# TODO 10: Añade gráficos de pérdida por época para ambos modelos
# ¿El gráfico de pérdida muestra una tendencia estable? ¿Existen caídas o aumentos repentinos y a qué podrían deberse?


# Complementario

In [ ]:
# Modificar el modelo para que prediga por palabras en lugar de caracteres.
# Cambia la lógica de preprocesamiento para trabajar con palabras en vez de caracteres.
# Ajusta el vocabulario para mapear palabras a índices (word_to_idx) y viceversa.
# Considera cómo cambiará SEQ_LENGTH, ya que ahora representará el número de palabras.
# Analiza si el texto generado es más coherente al trabajar a nivel de palabras.

In [ ]:
# Importamos las librerías necesarias
import torch
import numpy as np
import re
import torch.nn as nn

# Preprocesamos el texto
all_poems_text = ' '.join(poems_dict.values())
all_poems_text = all_poems_text.lower()
all_poems_text = re.sub(r'[^a-záéíóúüñ\s]', '', all_poems_text)

# Separamos el texto en palabras
words = all_poems_text.split()

# Creamos el vocabulario y los diccionarios
vocab = sorted(set(words))
vocab_size = len(vocab)
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for idx, word in enumerate(vocab)}

# Convertimos el texto en índices
text_as_int = [word_to_idx[word] for word in words]

# Definimos la longitud de las secuencias
SEQ_LENGTH = 10

# Creamos las secuencias y los targets
sequences = []
next_words = []
for i in range(0, len(text_as_int) - SEQ_LENGTH):
    sequences.append(text_as_int[i:i + SEQ_LENGTH])
    next_words.append(text_as_int[i + SEQ_LENGTH])

# Convertimos a tensores
X = torch.tensor(sequences, dtype=torch.long)
y = torch.tensor(next_words, dtype=torch.long)

print(f"Tamaño del vocabulario: {vocab_size}")
print(f"Número de secuencias: {X.size(0)}")

# Definimos el modelo RNN
class CustomizableRNN(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=512, num_layers=2, dropout=0.3, activation='tanh'):
        super(CustomizableRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Capa de embeddings: convierte palabras en vectores
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # LSTM para procesar secuencias
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        # Capa fully connected para la salida
        self.fc = nn.Linear(hidden_size, vocab_size)

        # Función de activación
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:
            self.activation = nn.Identity()

    def forward(self, x, h):
        # x es la entrada, h es el estado oculto
        x = self.embedding(x)
        out, h = self.lstm(x, h)
        out = self.activation(out[:, -1, :])
        out = self.fc(out)
        return out, h

    def init_hidden(self, batch_size):
        # Inicializamos el estado oculto y de memoria
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))

# Hiperparámetros
embed_size = 128
hidden_size = 512
num_layers = 2
dropout = 0.3
activation = 'tanh'

# Creamos el modelo
model = CustomizableRNN(vocab_size, embed_size, hidden_size, num_layers, dropout, activation)

# Definimos pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Usamos GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
X, y = X.to(device), y.to(device)

# Función de entrenamiento con precisión
def train(model, X, y, epochs=5, batch_size=64):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        for i in range(0, len(X) - batch_size, batch_size):
            inputs = X[i:i+batch_size]
            targets = y[i:i+batch_size]

            # Inicializamos el estado oculto
            h = model.init_hidden(batch_size)
            h = tuple([each.to(device) for each in h])

            optimizer.zero_grad()
            h = tuple([each.data for each in h])

            # Forward pass
            output, h = model(inputs, h)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Cálculo de precisión
            preds = output.argmax(dim=1)
            correct_predictions += (preds == targets).sum().item()
            total_samples += targets.size(0)

        avg_loss = total_loss / (len(X) // batch_size)
        accuracy = correct_predictions / total_samples
        print(f"Epoch [{epoch+1}/{epochs}], Pérdida promedio: {avg_loss:.4f}, Precisión: {accuracy:.4f}")

# Entrenamos el modelo
batch_size = 32
epochs = 30
train(model, X, y, epochs=epochs, batch_size=batch_size)

# Función para generar texto
def generate_text(model, seed_text, length=50):
    model.eval()
    generated = seed_text
    h = model.init_hidden(1)
    h = tuple([each.to(device) for each in h])

    seed_words = seed_text.lower().split()
    seed_indices = [word_to_idx.get(word, 0) for word in seed_words]
    input_seq = torch.tensor([seed_indices], dtype=torch.long).to(device)

    for _ in range(length):
        # Generamos la siguiente palabra
        output, h = model(input_seq, h)
        prob = output.softmax(dim=1).data
        word_idx = torch.multinomial(prob, 1).item()
        word = idx_to_word[word_idx]
        generated += ' ' + word

        # Actualizamos la secuencia de entrada
        input_seq = torch.tensor([[word_idx]], dtype=torch.long).to(device)

    return generated

Tamaño del vocabulario: 3852
Número de secuencias: 13992
Epoch [1/30], Pérdida promedio: 6.9718, Precisión: 0.0534
Epoch [2/30], Pérdida promedio: 6.1972, Precisión: 0.0546
Epoch [3/30], Pérdida promedio: 5.9394, Precisión: 0.0545
Epoch [4/30], Pérdida promedio: 5.6999, Precisión: 0.0611
Epoch [5/30], Pérdida promedio: 5.4233, Precisión: 0.0726
Epoch [6/30], Pérdida promedio: 5.1684, Precisión: 0.0799
Epoch [7/30], Pérdida promedio: 4.9028, Precisión: 0.0861
Epoch [8/30], Pérdida promedio: 4.6647, Precisión: 0.0910
Epoch [9/30], Pérdida promedio: 4.4826, Precisión: 0.0993
Epoch [10/30], Pérdida promedio: 4.2957, Precisión: 0.1104
Epoch [11/30], Pérdida promedio: 4.1561, Precisión: 0.1241
Epoch [12/30], Pérdida promedio: 3.8625, Precisión: 0.1512
Epoch [13/30], Pérdida promedio: 3.6117, Precisión: 0.1904
Epoch [14/30], Pérdida promedio: 3.3779, Precisión: 0.2283
Epoch [15/30], Pérdida promedio: 3.1350, Precisión: 0.2720
Epoch [16/30], Pérdida promedio: 3.0265, Precisión: 0.2908
Epoch [1

In [ ]:
# Generamos texto con el modelo entrenado
seed_text = "El viento susurra"
generated_poem = generate_text(model, seed_text, length=50)
print("\nPoema generado:\n")
print(generated_poem)



Poema generado:

El viento susurra casa ardían violetas las infecto largo jiménez sus lo claro le también se señor de los campos se también a la palabra buena del hombre del río ni eres tú le noche de quijote el aquel profesor cual la haces galería y caminar donde él es el vida verde por


In [ ]:
# ¿La generación por palabras afecta la fluidez del texto? ¿Qué ventajas o desventajas encuentras?
# ¿Hay sobreajuste? ¿Como lo solucionaria?